# Puzzle 1
--- Day 5: Cafeteria ---

As the forklifts break through the wall, the Elves are delighted to discover that there was a cafeteria on the other side after all.

You can hear a commotion coming from the kitchen. "At this rate, we won't have any time left to put the wreaths up in the dining hall!" Resolute in your quest, you investigate.

"If only we hadn't switched to the new inventory management system right before Christmas!" another Elf exclaims. You ask what's going on.

The Elves in the kitchen explain the situation: because of their complicated new inventory management system, they can't figure out which of their ingredients are fresh and which are spoiled. When you ask how it works, they give you a copy of their database (your puzzle input).

The database operates on ingredient IDs. It consists of a list of fresh ingredient ID ranges, a blank line, and a list of available ingredient IDs. For example:

```
3-5
10-14
16-20
12-18

1
5
8
11
17
32
```

The fresh ID ranges are inclusive: the range 3-5 means that ingredient IDs 3, 4, and 5 are all fresh. The ranges can also overlap; an ingredient ID is fresh if it is in any range.

The Elves are trying to determine which of the available ingredient IDs are fresh. In this example, this is done as follows:

    Ingredient ID 1 is spoiled because it does not fall into any range.
    Ingredient ID 5 is fresh because it falls into range 3-5.
    Ingredient ID 8 is spoiled.
    Ingredient ID 11 is fresh because it falls into range 10-14.
    Ingredient ID 17 is fresh because it falls into range 16-20 as well as range 12-18.
    Ingredient ID 32 is spoiled.

So, in this example, 3 of the available ingredient IDs are fresh.

Process the database file from the new inventory management system. How many of the available ingredient IDs are fresh?


In [1]:
test_input = '''3-5
10-14
16-20
12-18

1
5
8
11
17
32'''.split('\n')
test_fresh, test_inventory = test_input[:test_input.index('')], test_input[test_input.index('') + 1:]
test_fresh = [list(map(int, interval.split('-'))) for interval in test_fresh]
test_inventory = list(map(int, test_inventory))

In [2]:
with open('../data/day5.txt', 'r') as input_fle:
    input = input_fle.readlines()
    input = [line.rstrip('\n') for line in input]
    input_fresh, input_inventory = input[:input.index('')], input[input.index('') + 1:]
    input_fresh = [list(map(int, interval.split('-'))) for interval in input_fresh]
    input_inventory = list(map(int, input_inventory))

In [13]:
class CafeteriaInventory:
    fresh_ranges: list[list[int]]
    inventory: list[int]

    def __init__(self, fresh_ranges, inventory):
        self.fresh_ranges = fresh_ranges
        self.inventory = inventory

    def _in_range(self, interval: list[int], test: int) -> bool:
        return (test >= interval[0] and test <= interval[1])
    
    def is_fresh(self, inventory_item: int) -> bool:
        return (sum([self._in_range(interval, inventory_item) for interval in self.fresh_ranges]) > 0)
    
    def fresh_count(self) -> int:
        return sum([self.is_fresh(item) for item in self.inventory])
    
    #added for puzzle 2
    def total_fresh_ids(self) -> int:
        total_fresh = 0
        self.fresh_ranges.sort(key = lambda x: x[0]) #sort intervals by starting point
        cur_start, cur_end = self.fresh_ranges[0]
        for start, end in self.fresh_ranges[1:]:
            if start > cur_end +1: #then the intervals are disjoint, so we can "cash in" and start again
                total_fresh += (cur_end - cur_start + 1)
                cur_start, cur_end = start, end
            else: #then the intervals overlap, and we should consider their union
                cur_end = max(cur_end, end)
        total_fresh += (cur_end - cur_start + 1)
        return total_fresh

In [14]:
testCafe = CafeteriaInventory(test_fresh, test_inventory)
testCafe.fresh_count()
# 3
# success!

3

In [15]:
cafeteria = CafeteriaInventory(input_fresh, input_inventory)
cafeteria.fresh_count()
# 513
# success!

513

# Puzzle 2
--- Part Two ---

The Elves start bringing their spoiled inventory to the trash chute at the back of the kitchen.

So that they can stop bugging you when they get new inventory, the Elves would like to know all of the IDs that the fresh ingredient ID ranges consider to be fresh. An ingredient ID is still considered fresh if it is in any range.

Now, the second section of the database (the available ingredient IDs) is irrelevant. Here are the fresh ingredient ID ranges from the above example:

```
3-5
10-14
16-20
12-18
```

The ingredient IDs that these ranges consider to be fresh are 3, 4, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, and 20. So, in this example, the fresh ingredient ID ranges consider a total of 14 ingredient IDs to be fresh.

Process the database file again. How many ingredient IDs are considered to be fresh according to the fresh ingredient ID ranges?


In [17]:
testCafe.total_fresh_ids()
# 14
# success!

14

In [ ]:
cafeteria.total_fresh_ids()
# 339668510830757
# success!

339668510830757

# Reflection 

Puzzle 1 was easy enough. For puzzle 2 I tried to be dumb and got punished for it. My initial solution:

```
    #added for puzzle 2
    def total_fresh(self):
        all_fresh = set([idx for interval in self.fresh_ranges for idx in range(interval[0], interval[1]+1)])
        return len(all_fresh)  
```

This is *logically* correct, but computationally silly. We don't need to keep track of what the fresh IDs are, just how many we've seen. This worked fine for the test input and crashed my kernel for the actual input, which consists of very large integers and (I assume) was designed to fail for exactly this type of solution.

My final solution just counts how many IDs we must have; the key insight here was sorting the fresh intervals by their starting number, which enabled a much easier union-then-sum procedure than what I tried first (solving for conditions on start and end that would result in overlap, then finding the size of the overlap). 